In [ ]:
# -*- coding: utf-8 -*-
import random
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_exponential
from hashlib import md5
from openai import OpenAI
from tqdm import tqdm
import os
import sys
import json

# Принудительная установка кодировки UTF-8
os.environ['PYTHONIOENCODING'] = 'utf-8'
if hasattr(sys.stdout, 'reconfigure'):
    sys.stdout.reconfigure(encoding='utf-8')
  
print("✅ Библиотеки загружены с поддержкой UTF-8")


✅ Библиотеки загружены с поддержкой UTF-8


In [ ]:
# Настройка OpenAI с API ключом
api_key = "api key"

client = OpenAI(api_key=api_key)
print("✅ OpenAI клиент настроен")

NUM_DIALOGS = 100
OUTPUT_FILE = "real_estate_dialogs3.csv"


✅ OpenAI клиент настроен


In [11]:
# Этапы воронки продаж на русском языке
SALES_FUNNEL_STAGES = [
    {
        "name": "Установление контакта",
        "prompt": "Сгенери диалог холодного звонка менеджера по недвижимости. Стиль менеджера: {manager_style}. Настроение клиента: {client_mood}. Цель: договориться о встрече. Детали: {details}. 5-8 реплик каждый.",
        "criteria": {
            "установление_контакта": 5,
            "выявление_потребностей": 2,
            "презентация": 1,
            "работа_с_возражениями": 3,
            "закрытие": 1
        },
        "variations": {
            "manager_style": ["дружелюбный", "профессиональный", "настойчивый"],
            "client_mood": ["раздраженный", "равнодушный", "заинтересованный"],
            "details": ["ЖК 'Премиум'", "вторичка в центре", "загородный дом"]
        }
    },
    {
        "name": "Выявление потребностей",
        "prompt": "Диалог на встрече в офисе. Менеджер использует {technique}. Тип клиента: {client_type}. Ключевые параметры: {params}. 6-10 реплик каждый.",
        "criteria": {
            "установление_контакта": 3,
            "выявление_потребностей": 5,
            "презентация": 2,
            "работа_с_возражениями": 2,
            "закрытие": 1
        },
        "variations": {
            "technique": ["СПИН-вопросы", "открытые вопросы", "активное слушание"],
            "client_type": ["молодая семья", "инвестор", "пенсионер"],
            "params": ["бюджет 6-8 млн", "сроки 3 месяца", "ипотека готова"]
        }
    },
    {
        "name": "Презентация объекта",
        "prompt": "Диалог при показе недвижимости. Объект: {property_type}. Метод презентации: {method}. Первое возражение: {objection}. 7-9 реплик каждый.",
        "criteria": {
            "установление_контакта": 2,
            "выявление_потребностей": 3,
            "презентация": 5,
            "работа_с_возражениями": 3,
            "закрытие": 2
        },
        "variations": {
            "property_type": ["1-к квартира", "студия", "таунхаус"],
            "method": ["FAB метод", "демонстрация преимуществ", "сторителлинг"],
            "objection": ["мало света", "далеко от метро", "слишком высокие потолки"]
        }
    },
    {
        "name": "Работа с возражениями",
        "prompt": "Клиент серьезно возражает по теме: {objection}. Менеджер применяет технику: {technique}. Итог: {outcome}. 8-12 реплик каждый.",
        "criteria": {
            "установление_контакта": 2,
            "выявление_потребностей": 2,
            "презентация": 3,
            "работа_с_возражениями": 5,
            "закрытие": 3
        },
        "variations": {
            "objection": ["слишком дорого", "плохой район", "нужен ремонт"],
            "technique": ["LAER метод", "согласие+контраргумент", "перефразирование"],
            "outcome": ["клиент согласился", "просит скидку", "откладывает решение"]
        }
    },
    {
        "name": "Закрытие сделки",
        "prompt": "Финальная часть переговоров. Техника закрытия: {method}. Уровень давления: {pressure}. Результат встречи: {result}. 5-7 реплик каждый.",
        "criteria": {
            "установление_контакта": 3,
            "выявление_потребностей": 2,
            "презентация": 2,
            "работа_с_возражениями": 3,
            "закрытие": 5
        },
        "variations": {
            "method": ["альтернативный выбор", "прямое предложение", "искусственное ограничение"],
            "pressure": ["акция до конца недели", "есть другие покупатели", "мягкое подталкивание"],
            "result": ["успешная сделка", "мягкий отказ", "продажа доп. услуги"]
        }
    }
]

# Системный промпт на русском языке
SYSTEM_PROMPT = """Ты эксперт по продажам недвижимости. Генерируй УНИКАЛЬНЫЕ диалоги по следующим правилам:

1. Естественность: веди переписку как в мессенджере с клиентом
2. Конкретика: точные цифры (цена, метраж, сроки), учитывай российский рынок недвижимости  
3. Разнообразие: в 20% диалогов допускай ошибки менеджера для реалистичности
4. Воронка продаж: не пропускай ключевые этапы взаимодействия

Формат ответа:
Менеджер: [текст]
Клиент: [текст]  
Менеджер: [текст]
...и так далее"""

print("✅ Настройки диалогов загружены на русском языке")


✅ Настройки диалогов загружены на русском языке


In [12]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def generate_dialog(scenario):
    """Генерирует один уникальный диалог с поддержкой UTF-8"""
    params = {k: random.choice(v) for k, v in scenario["variations"].items()}
    prompt = scenario["prompt"].format(**params)
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # ПРАВИЛЬНАЯ МОДЕЛЬ!
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.8,
        frequency_penalty=1.5,
        presence_penalty=1.2
    )
    dialog = response.choices[0].message.content
    return dialog, params

def generate_dataset():
    """Генерирует уникальные диалоги на русском языке"""
    dataset = []
    hashes = set()
    
    print(f"🚀 Начинаем генерацию {NUM_DIALOGS} диалогов на русском языке...")
    
    with tqdm(total=NUM_DIALOGS, desc="Генерация диалогов", ncols=100) as pbar:
        attempts = 0
        max_attempts = NUM_DIALOGS * 3  # Защита от бесконечного цикла
        
        while len(dataset) < NUM_DIALOGS and attempts < max_attempts:
            attempts += 1
            try:
                scenario = random.choice(SALES_FUNNEL_STAGES)
                dialog, params = generate_dialog(scenario)
                
                # Хэшируем с учетом UTF-8
                dialog_hash = md5(dialog.encode('utf-8')).hexdigest()
                
                if dialog_hash not in hashes:  # Проверка на уникальность
                    hashes.add(dialog_hash)
                    dataset.append({
                        "text": dialog,
                        "stage": scenario["name"],
                        **scenario["criteria"],
                        **params
                    })
                    pbar.update(1)
                    
            except Exception as e:
                print(f"⚠️ Ошибка при генерации диалога: {e}")
                continue
    
    return pd.DataFrame(dataset)

print("✅ Функции генерации готовы к работе")


✅ Функции генерации готовы к работе


In [13]:
# Запускаем генерацию диалогов
print("🎯 Начинаем генерацию диалогов...")

df = generate_dataset()
df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

print(f"✅ Сохранено {len(df)} уникальных диалогов в файл {OUTPUT_FILE}")
print(f"📊 Распределение по этапам:")
print(df['stage'].value_counts())
print(f"\n💾 Размер файла: {df.shape}")
print(f"📝 Пример диалога:")
print("=" * 50)
print(df.iloc[0]['text'][:500] + "...")


🎯 Начинаем генерацию диалогов...
🚀 Начинаем генерацию 100 диалогов на русском языке...


Генерация диалогов: 100%|█████████████████████████████████████████| 100/100 [33:04<00:00, 19.85s/it]

✅ Сохранено 100 уникальных диалогов в файл real_estate_dialogs3.csv
📊 Распределение по этапам:
stage
Презентация объекта       26
Работа с возражениями     24
Выявление потребностей    19
Закрытие сделки           17
Установление контакта     14
Name: count, dtype: int64

💾 Размер файла: (100, 19)
📝 Пример диалога:
Менеджер: Добрый день! Рад вас видеть. Как настроение? 

Клиент: Добрый день! Все отлично, спасибо. Мы очень надеемся найти подходящую квартиру.

Менеджер: Это замечательно! Какие основные критерии для вас важны при выборе жилья?

Клиент: Нам нужна квартира с двумя спальнями и в районе с хорошей инфраструктурой — магазины, школы рядом. Площадь около 70-80 квадратов.

Менеджер: Понял вас. У нас есть несколько интересных вариантов в пределах 7-9 миллионов рублей на эту площадь. Вам важно также нал...
